In [1]:
import pandas as pd
import numpy as np
from Utils import *

In [2]:
data_folder = 'C:\\Users\\amoha\\Detect-Sleep-States-CMI-data\\'
series = "train_series.parquet"
events = "train_events.csv"


train_series = pd.read_parquet(data_folder+series)
train_events = pd.read_csv(data_folder+events)


In [3]:
train_series = train_series[:100000]

In [4]:
# null_values = train_events.isnull()
# null_counts = null_values.sum()
# Title = "Number of Null Values in Each Column of events"
# plot(null_counts,"bar",Title)

# null_values = train_series.isnull()
# null_counts = null_values.sum()
# Title = "Number of Null Values in Each Column of series"
# plot(null_counts,"bar",Title)

In [5]:
train_series["date"] = train_series["timestamp"].apply(lambda x: x[:10])
train_series["hour"] = train_series["timestamp"].apply(lambda x: x[11:16])


cleaned_train = train_events.dropna(subset=['timestamp']).copy()


cleaned_train["date"] = cleaned_train["timestamp"].apply(lambda x: x[:10])
cleaned_train["star_hr"] = cleaned_train["timestamp"].apply(lambda x: x[11:16])
cleaned_train["end_hr"] = cleaned_train["timestamp"].apply(lambda x: x[20:])
cleaned_train["end_hr"] = cleaned_train["end_hr"].apply(lambda x: x[:2] + ':' + x[2:])



num_days = len(train_series.date.unique())
print(f"There are {num_days} days of data in the dataset")

cleaned_train


There are 7 days of data in the dataset


,series_id,night,event,step,timestamp,date,star_hr,end_hr
0,038441c925bb,1,onset,4992.0,2018-08-14T22:26:00-0400,2018-08-14,22:26,04:00
1,038441c925bb,1,wakeup,10932.0,2018-08-15T06:41:00-0400,2018-08-15,06:41,04:00
2,038441c925bb,2,onset,20244.0,2018-08-15T19:37:00-0400,2018-08-15,19:37,04:00
3,038441c925bb,2,wakeup,27492.0,2018-08-16T05:41:00-0400,2018-08-16,05:41,04:00
4,038441c925bb,3,onset,39996.0,2018-08-16T23:03:00-0400,2018-08-16,23:03,04:00
...,...,...,...,...,...,...,...,...
14501,fe90110788d2,32,wakeup,547152.0,2017-09-05T09:26:00-0400,2017-09-05,09:26,04:00
14502,fe90110788d2,33,onset,556560.0,2017-09-05T22:30:00-0400,2017-09-05,22:30,04:00
14503,fe90110788d2,33,wakeup,560604.0,2017-09-06T04:07:00-0400,2017-09-06,04:07,04:00
14504,fe90110788d2,34,onset,574620.0,2017-09-06T23:35:00-0400,2017-09-06,23:35,04:00


In [6]:
cleaned_train = cleaned_train.sort_values(by='night')
cleaned_train = cleaned_train.reset_index(drop=True)

In [7]:
len(cleaned_train.date.unique())

700

In [8]:
onset_data = cleaned_train[cleaned_train.event == "onset"]
wakeup_data = cleaned_train[cleaned_train.event == "wakeup"]
onset_data = onset_data.reset_index()
wakeup_data = wakeup_data.reset_index()

In [9]:
onset_data['date'] = pd.to_datetime(onset_data['date']).copy()
onset_data['star_hr'] = pd.to_datetime(onset_data['star_hr']).dt.time
onset_data['end_hr'] = pd.to_datetime(onset_data['end_hr']).dt.time


train_series['timestamp'] = pd.to_datetime(train_series['timestamp']).dt.time
train_series['date'] = pd.to_datetime(train_series['date']).dt.time

train_series['hour'] = pd.to_datetime(train_series['hour']).dt.time

In [15]:
onset = []
for idx,date in enumerate(onset_data.date):
    start = onset_data.star_hr[idx]
    end = onset_data.end_hr[idx]
    series_id = onset_data.series_id[idx]
    extracted_subset = train_series[
        (train_series['series_id'] == series_id) &
        (train_series['hour'] >= start) &    # Start hour condition
        (train_series['hour'] <= end)       # End hour condition
    ]
    onset.append(extracted_subset)